In [2]:
import os
import nltk
from nltk.corpus import stopwords
from gensim.utils import simple_preprocess
from gensim.corpora import Dictionary
from gensim.models import LdaModel

# -------------------------------
# 1️⃣ Paramètres
# -------------------------------
data_path = "../../data/txt/"  # chemin vers ton corpus
n_topics = 3  # nombre de thèmes à extraire
n_top_words = 10  # nombre de mots par thème
passes = 10  # nombre de passes d'apprentissage sur le corpus

# -------------------------------
# 2️⃣ Chargement du corpus
# -------------------------------
files = [f for f in sorted(os.listdir(data_path)) if f.endswith(".txt")]
texts = [open(os.path.join(data_path, f), "r", encoding="utf-8").read() for f in files]
print(f"{len(files)} fichiers chargés dans le corpus")

# -------------------------------
# 3️⃣ Prétraitement : tokenisation + suppression stopwords
# -------------------------------
nltk.download('stopwords')
stop_words = stopwords.words('english')

def preprocess(text):
    return [token for token in simple_preprocess(text) if token not in stop_words]

processed_texts = [preprocess(t) for t in texts]

# -------------------------------
# 4️⃣ Création du dictionnaire et du corpus
# -------------------------------
dictionary = Dictionary(processed_texts)
dictionary.filter_extremes(no_below=5, no_above=0.5)  # filtre mots trop rares ou trop fréquents
corpus = [dictionary.doc2bow(text) for text in processed_texts]

# -------------------------------
# 5️⃣ LDA avec LdaModel
# -------------------------------
lda = LdaModel(corpus=corpus,
               num_topics=n_topics,
               id2word=dictionary,
               passes=passes,
               random_state=42)

# -------------------------------
# 6️⃣ Affichage des thèmes
# -------------------------------
for topic_idx, topic in lda.show_topics(formatted=False, num_words=n_top_words):
    top_words = [word for word, prob in topic]
    print(f"\nThème {topic_idx + 1}: {', '.join(top_words)}")


987 fichiers chargés dans le corpus


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!



Thème 1: art, mme, el, dc, père, moi, ct, lo, famille, prince

Thème 2: tél, da, ea, ena, lo, mme, travailleurs, dc, socialiste, socialistes

Thème 3: milliards, congo, dc, dollars, bretagne, soviétique, travailleurs, britannique, décembre, banque


In [4]:
import os
import nltk
from nltk.corpus import stopwords
from gensim.utils import simple_preprocess
from gensim.corpora import Dictionary
from gensim.models import LdaModel # Utilisation du modèle natif plus stable

# -------------------------------
# 1️⃣ Paramètres (Ajustés selon les sources)
# -------------------------------
data_path = "../../data/txt/" 
# Les sources recommandent 280 thèmes pour une analyse fine [3]
n_topics = 20 # Augmentez à 280 si vous avez un gros corpus
n_top_words = 20 # Standard utilisé par les auteurs [6]

# -------------------------------
# 2️⃣ Chargement et Prétraitement
# -------------------------------
# Téléchargement des ressources françaises
nltk.download('stopwords')
# CRUCIAL : Utiliser le français pour l'Encyclopédie [1]
stop_words = stopwords.words('french') 

def preprocess(text):
    # Les auteurs recommandent de filtrer les mots fonctionnels [3]
    return [token for token in simple_preprocess(text) if token not in stop_words and len(token) > 2]

# Chargement (gestion des erreurs d'encodage fréquente sur les textes anciens)
texts = []
for f in sorted(os.listdir(data_path)):
    if f.endswith(".txt"):
        with open(os.path.join(data_path, f), "r", encoding="utf-8", errors='ignore') as file:
            texts.append(preprocess(file.read()))

# -------------------------------
# 3️⃣ Création du dictionnaire et du modèle
# -------------------------------
dictionary = Dictionary(texts)
# Filtrage : les auteurs excluent les mots trop fréquents ou rares [9]
dictionary.filter_extremes(no_below=5, no_above=0.5)
corpus = [dictionary.doc2bow(text) for text in texts]

# Entraînement du modèle LDA
lda = LdaModel(corpus=corpus, num_topics=n_topics, id2word=dictionary, passes=10)

# -------------------------------
# 4️⃣ Affichage (comparable aux Tables 1 & 2 des sources)
# -------------------------------
for topic_idx in range(n_topics):
    # Récupération des 20 mots-clés [6]
    words = lda.show_topic(topic_idx, topn=n_top_words)
    topic_words = [word for word, prob in words]
    print(f"\nThème {topic_idx + 1} (Disours potentiel) : {', '.join(topic_words)}")

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!



Thème 1 (Disours potentiel) : britannique, bretagne, soviétique, avril, nations, décembre, communistes, américain, américains, washington, américaine, juillet, ouest, congrès, berlin, communiste, déclaration, septembre, moscou, août

Thème 2 (Disours potentiel) : milliards, banque, dollars, taux, valeurs, actions, hausse, fonds, bourse, entreprises, augmentation, crédit, sociétés, baisse, décembre, investissements, capitaux, juin, capital, niveau

Thème 3 (Disours potentiel) : tél, charleroi, vente, vendre, décembre, beau, meubles, adresser, belle, bilan, cardinal, mme, ecr, adr, choix, louer, rossel, père, louis, fabrique

Thème 4 (Disours potentiel) : sport, division, sieur, membre, daring, iii, ena, film, sortant, namur, aller, antwerp, gand, gantoise, tirlemont, standard, malines, charlerol, tournai, fléron

Thème 5 (Disours potentiel) : esl, socialistes, reine, laurent, communistes, élait, élé, reuter, libéraux, cetle, tribunal, pré, sœur, new, york, discussion, renée, élections,

In [1]:
import os
import nltk
from nltk.corpus import stopwords
from gensim.corpora import Dictionary
from gensim.models import LdaModel
from gensim.utils import simple_preprocess

# -------------------------------
# 1️⃣ Paramètres basés sur les sources
# -------------------------------
# Les auteurs ont testé 280, 300, 330 et 360 thèmes [1].
# Ils ont conclu que le modèle à 280 thèmes était le plus cohérent [2].
n_topics = 10
# L'analyse repose sur les 20 mots les plus significatifs par thème [2].
n_top_words = 20 

# -------------------------------
# 2️⃣ Chargement et Prétraitement
# -------------------------------
# Chemin vers votre fichier spécifique
file_path = "../../data/tmp/corpus_clean2.txt"

# Les sources soulignent l'importance des noms (substantifs) [3].
# Pour simplifier, nous utilisons ici une suppression rigoureuse des stopwords.
nltk.download('stopwords')
stop_words = stopwords.words('french') # Crucial pour l'Encyclopédie [1, 4]

def preprocess(text):
    # Les auteurs filtrent les mots fonctionnels (stopwords) [1].
    # Ils suggèrent de ne garder que les noms et noms propres [5].
    tokens = simple_preprocess(text)
    return [t for t in tokens if t not in stop_words and len(t) > 2]

print("Chargement du corpus...")
with open(file_path, "r", encoding="utf-8") as f:
    # On traite chaque ligne comme un article/document indépendant [6, 7].
    processed_texts = [preprocess(line) for line in f]

# -------------------------------
# 3️⃣ Création du modèle (LDA)
# -------------------------------
# Création du dictionnaire et du format "Bag of Words" (sac de mots) [8].
dictionary = Dictionary(processed_texts)
# Filtrage des mots trop rares ou trop fréquents comme dans PhiloMine [7].
#dictionary.filter_extremes(no_below=2, no_above=0.8)
corpus = [dictionary.doc2bow(text) for text in processed_texts]

print(f"Entraînement du modèle avec {n_topics} thèmes...")
# Bien que les auteurs utilisent MALLET [1], le modèle LdaModel 
# de Gensim est une alternative moderne produisant des résultats comparables.
lda = LdaModel(corpus=corpus, num_topics=n_topics, id2word=dictionary, passes=10)

# -------------------------------
# 4️⃣ Analyse des résultats
# -------------------------------
# Affichage des thèmes pour identifier les "discours" [2, 8].
for topic_idx in range(n_topics):
    words = lda.show_topic(topic_idx, topn=n_top_words)
    topic_words = [word for word, prob in words]
    print(f"\nThème {topic_idx + 1} : {', '.join(topic_words)}")

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Chargement du corpus...
Entraînement du modèle avec 10 thèmes...

Thème 1 : doute, cent, surtout, donner, banque, produits, sait, grands, six, seul, etc, réunion, août, services, groupe, française, mai, ouvriers, semble, dix

Thème 2 : britannique, sud, semble, voici, ouvriers, font, ensemble, août, grands, doivent, pourquoi, politiques, nombreux, février, américain, congrès, donner, chose, seul, anglais

Thème 3 : côté, administration, occasion, sud, conditions, agit, arrêté, banque, services, régime, produits, grands, public, grandes, surtout, américains, police, internationale, groupe, petit

Thème 4 : produits, etc, services, voix, groupe, mettre, française, sait, certaines, possible, intérêt, ouvriers, semble, grandes, public, britannique, main, dépenses, étrangères, mai

Thème 5 : septembre, voici, services, américaine, grands, petit, dollars, pierre, police, ancien, tour, socialiste, moyen, mieux, souvent, public, reçu, personnel, réunion, étrangères

Thème 6 : public, doute, di